# Relational Implementaion

In [21]:
import pymysql
import json
import pandas as pd

cnx = pymysql.connect(host='localhost',
						user='root',
						password='Lz19940305sql',
						db='lahman2017',
						charset='utf8',
						cursorclass=pymysql.cursors.DictCursor)

cursor = cnx.cursor()
query = "SELECT \
Batting.playerID, \
(SELECT People.nameFirst FROM People WHERE People.playerID=Batting.playerID) as first_name, \
(SELECT People.nameLast FROM People WHERE People.playerID=Batting.playerID) as last_name, \
sum(Batting.h)/sum(batting.ab) as career_average, \
sum(Batting.h) as career_hits, \
sum(Batting.ab) as career_at_bats,\
min(Batting.yearID) as first_year, \
max(Batting.yearID) as last_year \
FROM \
Batting \
GROUP BY \
playerId \
HAVING \
career_at_bats > 200 AND last_year >= 1960 \
ORDER BY \
career_average DESC \
LIMIT 10;"
cursor.execute(query)
r = cursor.fetchall()
r = pd.DataFrame.from_dict(r)
r

,career_at_bats,career_average,career_hits,first_name,first_year,last_name,last_year,playerID
0,7706,0.3444,2654,Ted,1939,Williams,1960,willite01
1,9288,0.3382,3141,Tony,1982,Gwynn,2001,gwynnto01
2,10972,0.3308,3630,Stan,1941,Musial,1963,musiast01
3,347,0.3285,114,Trea,2015,Turner,2016,turnetr01
4,9180,0.3279,3010,Wade,1982,Boggs,1999,boggswa01
5,9315,0.3278,3053,Rod,1967,Carew,1985,carewro01
6,7853,0.3208,2519,Miguel,2003,Cabrera,2016,cabremi01
7,7244,0.3181,2304,Kirby,1984,Puckett,1995,puckeki01
8,8155,0.3176,2590,Vladimir,1996,Guerrero,2011,guerrvl01
9,9454,0.3173,3000,Roberto,1955,Clemente,1972,clemero01


# CSV Implementation

In [13]:
import csv          # Python package for reading and writing CSV files.
import json
import sys,os

In [23]:
rel_path = os.path.realpath('./Data')
data_dir = rel_path + "/" + "Batting.csv"
batting_table = []
with open(data_dir) as csv_file:
    csv_reader = csv.DictReader(csv_file, delimiter=',')
    for row in csv_reader:
        batting_table.append(row)

In [16]:
data_dir = rel_path + "/" + "People.csv"
people_table = []
with open(data_dir) as csv_file:
    csv_reader = csv.DictReader(csv_file, delimiter=',')
    for row in csv_reader:
        people_table.append(row)

In [34]:
playerID = {}
for item in batting_table:
    if item["playerID"] not in playerID.keys():
        playerID[item["playerID"]] = []
        playerID[item["playerID"]].append(item)
    else:
        playerID[item["playerID"]].append(item)

In [37]:
agg_dict = {}
for k, v in playerID.items():
    ab = []
    h = []
    year = []
    for item in v:
        ab.append(float(item['AB']))
        h.append(float(item['H']))
        year.append(int(item['yearID']))
    if sum(ab) == 0:
        p = None
    else:
        p = sum(h) / sum(ab)
    agg_dict[k] = {'career_average': p, "career_at_bats": sum(ab), 
                   "career_hits": sum(h), "last_year": max(year), "first_year": min(year)}
    

In [38]:
condition_dict = {}
for k, v in agg_dict.items():
    if v['last_year'] >= 1960 and v['career_at_bats'] > 200:
        condition_dict[k] = v

In [39]:
career_at_bats = []
career_hits = []
career_average = []
last_year = []
first_year = []
for v in condition_dict.values():
    career_at_bats.append(v['career_at_bats'])
    career_hits.append(v['career_hits'])
    career_average.append(v['career_average'])
    last_year.append(v['last_year'])
    first_year.append(v['first_year'])

In [42]:
lastname = []
firstname = []
for ID in condition_dict.keys():
    for row in people_table:
        if row["playerID"] == ID:
            lastname.append(row["nameLast"])
            firstname.append(row["nameFirst"])


In [40]:
df = pd.DataFrame()
df["playerID"] = condition_dict.keys()
df["career_at_bats"] = career_at_bats
df["career_hits"] = career_hits
df["career_average"] = career_average
df["last_year"] = last_year
df["first_year"] = first_year

In [43]:
df["last_name"] = lastname
df["first_name"] = firstname

In [46]:
df = df.sort_values(['career_average'], ascending=False)

In [47]:
df.head(10)

,playerID,career_at_bats,career_hits,career_average,last_year,first_year,last_name,first_name
1,willite01,7706.0,2654.0,0.344407,1960,1939,Williams,Ted
1690,gwynnto01,9288.0,3141.0,0.338178,2001,1982,Gwynn,Tony
5,musiast01,10972.0,3630.0,0.330842,1963,1941,Musial,Stan
3838,turnetr01,347.0,114.0,0.328530,2016,2015,Turner,Trea
1680,boggswa01,9180.0,3010.0,0.327887,1999,1982,Boggs,Wade
844,carewro01,9315.0,3053.0,0.327751,1985,1967,Carew,Rod
2995,cabremi01,7853.0,2519.0,0.320769,2016,2003,Cabrera,Miguel
1839,puckeki01,7244.0,2304.0,0.318056,1995,1984,Puckett,Kirby
2545,guerrvl01,8155.0,2590.0,0.317597,2011,1996,Guerrero,Vladimir
207,clemero01,9454.0,3000.0,0.317326,1972,1955,Clemente,Roberto
